# Test of looping over $\beta$-VAE to detect and classify outliers

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm

from pathlib import Path

In [2]:
from magnetics_diagnostic_analysis.project_vae.setting_vae import config



Choosen device = cuda:0


In [3]:
import torch
from torch import nn

from magnetics_diagnostic_analysis.ml_tools.pytorch_device_selection import print_torch_info
print_torch_info()


Torch version?  2.4.1+cu121
Cuda?           True

GPU number : 2
GPU 0: Tesla T4
GPU 1: Tesla T4


In [4]:
suffix = "vae"

### 1. Create dataset and DataLoader

I took the decision that one data sample will be : all the time values of one shot and for all diagnostics. It will be easy after, to reduce to one diagnostic only (It wouldn't habe been the case if we wanted to use all diagnostics for one timestep -> there reduce to one diagnostic just give us one number and that is to small).

As all shots own different lenghts, we are going to use LSTM unit in entry of our VAE. This LSTM unit is combined with padded sequence and have masking behaviour.

Thus, after the LSTM, we will have a constant size tensor (the LSTM hidden state) that we can use in our VAE.

Consideration:

We want our model to be robust to any different size during testing time.

Thus, we are going to find the max_lenght for each batch size in the dataloader.

And thanks to the two functions `pack_padded_sequence`, `pad_packed_sequence`, the LSTM is aware of the true lenght of each sequence and use masking.

In [5]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

In [6]:
path = Path().absolute().parent.parent / "data/preprocessed/mscred/data_magnetics_mscred_cleaned.nc"
data_all = xr.open_dataset(path)
data_all

<xarray.Dataset> Size: 10GB
Dimensions:                                   (time: 12877819,
                                               b_field_tor_probe_saddle_voltage_channel: 9,
                                               b_field_pol_probe_obr_channel: 18,
                                               b_field_pol_probe_ccbv_channel: 37,
                                               flux_loop_channel: 14,
                                               b_field_pol_probe_obv_channel: 17)
Coordinates:
  * b_field_tor_probe_saddle_voltage_channel  (b_field_tor_probe_saddle_voltage_channel) <U15 540B ...
  * time                                      (time) float64 103MB -0.0692 .....
  * b_field_pol_probe_obr_channel             (b_field_pol_probe_obr_channel) <U9 648B ...
  * b_field_pol_probe_ccbv_channel            (b_field_pol_probe_ccbv_channel) <U10 1kB ...
  * flux_loop_channel                         (flux_loop_channel) <U12 672B '...
  * b_field_pol_probe_obv_channel             (b_field_pol_probe_obv_channel) <U9 612B ...
Data variables:
    b_field_tor_probe_saddle_voltage          (time, b_field_tor_probe_saddle_voltage_channel) float64 927MB ...
    b_field_pol_probe_obr_field               (time, b_field_pol_probe_obr_channel) float64 2GB ...
    ip                                        (time) float64 103MB ...
    b_field_pol_probe_ccbv_field              (time, b_field_pol_probe_ccbv_channel) float64 4GB ...
    flux_loop_flux                            (time, flux_loop_channel) float64 1GB ...
    b_field_pol_probe_obv_field               (time, b_field_pol_probe_obv_channel) float64 2GB ...
    shot_index                                (time) int64 103MB ...

In [7]:
def find_seq_length(data: xr.Dataset) -> np.ndarray:
    # Find the length of each sequence in the dataset
    seq_indices = data['shot_index'].values
    return np.bincount(seq_indices)

lengths = find_seq_length(data_all)
print(data_all['shot_index'].values)
print(lengths)
print("Sequence lengths:", lengths.shape)
print("None null values:", lengths[lengths > 0].shape)
print("Index where null values:", np.where(lengths == 0)[0])
print(lengths[550: 560])
print(lengths[2870: 2880])

[   0    0    0 ... 6117 6117 6117]
[2084 1989 2230 ... 3153 2089 1947]
Sequence lengths: (6118,)
None null values: (6116,)
Index where null values: [ 557 2874]
[2116 2016 1970  855 1634 1663 2193    0 2193  891]
[2082 1636 1648  982    0 1875 1699 2439 5635 1749]


In [8]:
class MultivariateTimeSerieDataset(Dataset):
    def __init__(self, data: xr.Dataset, n_chan_to_keep: int = 4, n_subsample: int = 10, max_length: int = 3000):
        # Group data by shot_index
        self.shot_indices = data['shot_index'].values
        self.unique_shots = np.unique(self.shot_indices)
        
        # Precompute sequences for each shot index
        self.sequences = {}
        for shot in self.unique_shots:
            mask = self.shot_indices == shot
            shot_data = []
            for var in data.data_vars:
                if var == 'shot_index':
                    continue

                if data[var].ndim == 1:
                    var_data = data[var].values[mask][:, np.newaxis]
                    if len(var_data) > max_length:
                        var_data = var_data[:max_length]

                else:
                    var_data = data[var].values[mask]
                    if var_data.shape[1] > n_chan_to_keep:
                        var_data = var_data[:, :n_chan_to_keep]
                    if len(var_data) > max_length:
                        var_data = var_data[:max_length]
                        
                var_data = var_data[::n_subsample]
                shot_data.append(var_data)
            self.sequences[shot] = np.concatenate(shot_data, axis=1)      # axis=1 => along features dimension
        
        self.lengths = {shot: len(self.sequences[shot]) for shot in self.unique_shots}

    def __len__(self):
        return len(self.unique_shots)
    
    def __getitem__(self, idx):
        shot = self.unique_shots[idx]
        return self.sequences[shot], self.lengths[shot]
    
class OneVariableTimeSerieDataset(Dataset):
    def __init__(self, data: xr.Dataset, var_name: str = "ip", chan_to_keep: None | int = 1, n_subsample: int = 12, max_length: int = 3000):
        # Group data by shot_index
        self.shot_indices = data['shot_index'].values
        self.unique_shots = np.unique(self.shot_indices)
        
        # Precompute sequences for each shot index
        self.sequences = {}
        for shot in self.unique_shots:
            mask = self.shot_indices == shot
            data_var = data[var_name].values[mask]
            if data_var.ndim > 1:
                data_var = data_var[:, chan_to_keep]
            if len(data_var) > max_length:
                data_var = data_var[:max_length] 
            data_var = data_var[::n_subsample]
            self.sequences[shot] = data_var[:, np.newaxis]  # Add feature dimension

        self.lengths = {shot: len(self.sequences[shot]) for shot in self.unique_shots}

    def __len__(self):
        return len(self.unique_shots)
    
    def __getitem__(self, idx):
        shot = self.unique_shots[idx]
        return self.sequences[shot], self.lengths[shot]

In [9]:
def create_datasets(
    data: xr.Dataset,
    set_separation: int = 12000,
    total_length: int = 3500,
    rd_seed: int = 42
) -> tuple[Dataset]:
    """
    Create train, validation and test data loaders from time series data
    
    Args:
        data: xarray Dataset with shot_index variable
        batch_size: batch size for data loaders
        set_separation: boundarie between train and test sets
        device: device to load data on
    
    Returns:
        train_loader, valid_loader, test_loader: DataLoader objects
    """
    data = data.isel(time=slice(0, total_length))

    shot_indices = data['shot_index'].values
    seq_lengths = np.bincount(shot_indices)
    unique_shots = np.unique(shot_indices)
    available_shots = unique_shots.copy()

    print("unique shots:", unique_shots.shape)

    rng = np.random.default_rng(rd_seed)

    test_shot_indices = []
    cumulative_time = 0

    while (cumulative_time < total_length - set_separation):
        shot_idx = rng.choice(available_shots, size=1, replace=False)[0]
        available_shots = available_shots[available_shots != shot_idx]

        shot_length = seq_lengths[shot_idx]
        test_shot_indices.append(shot_idx)
        cumulative_time += shot_length

    train_shot_indices = list(available_shots)
    print("Train shots:", len(train_shot_indices), "Test shots:", len(test_shot_indices))
    assert len(train_shot_indices) + len(test_shot_indices) == len(unique_shots), "Some shots are missing in the split"

    train_mask = np.isin(shot_indices, train_shot_indices)
    test_mask = np.isin(shot_indices, test_shot_indices)
    print("Train samples:", train_mask.sum(), "Test samples:", test_mask.sum())

    # Create datasets for each split
    train_dataset = OneVariableTimeSerieDataset(data.isel(time=train_mask), var_name="ip", chan_to_keep=None, n_subsample=25, max_length=3000)
    test_dataset = OneVariableTimeSerieDataset(data.isel(time=test_mask), var_name="ip", chan_to_keep=None, n_subsample=25, max_length=3000)
    # train_dataset = MultivariateTimeSerieDataset(data.isel(time=train_mask), n_chan_to_keep=4, n_subsample=12, max_length=3000)
    # test_dataset = MultivariateTimeSerieDataset(data.isel(time=test_mask), n_chan_to_keep=4, n_subsample=12, max_length=3000)

    return train_dataset, test_dataset

In [ ]:
# Average running time: 5min 30sec

train_set, test_set = create_datasets(data_all, set_separation=config.SET_SEPARATION, total_length=config.DATA_NUMBER, rd_seed=config.SEED)

print("Training set size:", len(train_set))
print("Testing set size:", len(test_set))
print(train_set)

In [ ]:
path_train = config.DIR_PREPROCESSED_DATA / f"dataset_raw_vae_train.pt"
path_test = config.DIR_PREPROCESSED_DATA / f"dataset_raw_vae_test.pt"
if not path_train.exists():
    torch.save(train_set, path_train)
    print(f"Saved dataset to {path_train}")
if not path_test.exists():
    torch.save(test_set, path_test)
    print(f"Saved dataset to {path_test}")


In [10]:
path_train = config.DIR_PREPROCESSED_DATA / f"dataset_ip_vae_train.pt"
path_test = config.DIR_PREPROCESSED_DATA / f"dataset_ip_vae_test.pt"
train_set = torch.load(path_train)
test_set = torch.load(path_test)

/tmp/ipykernel_1007415/2784269247.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_set = torch.load(path_train)
/tmp/ipykernel_1007415/2784269247.py:4: FutureWarnin

In [11]:
def pad_sequences_smartly(batch):
    """Custom collate function to pad sequences to max length in batch"""
    sequences, lengths = zip(*batch)
    
    # Convert sequences to tensors
    sequence_tensors = [torch.from_numpy(seq).float() for seq in sequences]
    padded_sequences = pad_sequence(
        sequence_tensors, 
        batch_first=True, 
        padding_value=0.0
    )
    length_tensor = torch.tensor(lengths, dtype=torch.long)
    
    return padded_sequences, length_tensor

## 2. Model LSTM-$\beta$-VAE implementation

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be divisible by number of heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Projection layers for queries, keys and values
        self.q_proj = nn.Linear(input_dim, embed_dim)
        self.k_proj = nn.Linear(input_dim, embed_dim)
        self.v_proj = nn.Linear(input_dim, embed_dim)
        
        self.output_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.scale = np.sqrt(self.head_dim)
        
    def forward(self, query, key, value, key_padding_mask=None):
        """
        query: [batch_size, query_len, input_dim]
        key: [batch_size, key_len, input_dim]
        value: [batch_size, value_len, input_dim]
        key_padding_mask: [batch_size, key_len] (True for padding positions)
        """
        batch_size = query.size(0)
        query_len = query.size(1)
        key_len = key.size(1)
        
        # Project queries, keys and values
        Q = self.q_proj(query)  # [batch_size, query_len, embed_dim]
        K = self.k_proj(key)    # [batch_size, key_len, embed_dim]
        V = self.v_proj(value)  # [batch_size, value_len, embed_dim]
        
        # Reshape for multi-head attention
        Q = Q.view(batch_size, query_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, key_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, key_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Calculate attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale  # [batch_size, num_heads, query_len, key_len]
        
        # Apply mask if provided
        if key_padding_mask is not None:
            # Expand mask to [batch_size, num_heads, query_len, key_len]
            mask = key_padding_mask.unsqueeze(1).unsqueeze(2).expand(-1, self.num_heads, query_len, -1)
            scores = scores.masked_fill(mask, float('-inf'))
        
        # Apply softmax to get attention weights
        attn_weights = nn.functional.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Apply attention to values
        attn_output = torch.matmul(attn_weights, V)  # [batch_size, num_heads, query_len, head_dim]
        
        # Concatenate heads and put through final linear layer
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, query_len, self.embed_dim)
        attn_output = self.output_proj(attn_output)
        
        return attn_output, attn_weights
    
class MultiHeadAttention2(nn.Module):
    def __init__(self, dim, num_heads=8):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        
        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.out_proj = nn.Linear(dim, dim)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        
        # Projections linéaires
        Q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim)
        K = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim)
        V = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim)
        
        # Calcul d'attention par tête
        scores = torch.einsum('bqhd,bkhd->bhqk', Q, K) / math.sqrt(self.head_dim)
        attention = torch.softmax(scores, dim=-1)
        out = torch.einsum('bhqk,bkhd->bqhd', attention, V)
        
        # Concaténation et projection finale
        out = out.contiguous().view(batch_size, seq_len, -1)
        return self.out_proj(out)

In [ ]:
class SafeAttentionEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads=4):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, 
                           batch_first=True, bidirectional=True)
        
        # Attention sur la sortie LSTM
        self.attention = MultiHeadAttention(
            input_dim=hidden_dim * 2,
            embed_dim=hidden_dim * 2, 
            num_heads=num_heads
        )
        
        # Couches pour produire mean et logvar
        self.to_mean = nn.Linear(hidden_dim * 2, latent_dim)
        self.to_logvar = nn.Linear(hidden_dim * 2, latent_dim)
        
    def forward(self, x, lengths):
        # 1. LSTM
        packed_input = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        lstm_output, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        # 2. Masque pour l'attention
        mask = torch.arange(x.size(1), device=x.device)[None, :] >= lengths[:, None]
        
        # 3. Auto-attention sur la sortie LSTM
        attn_output, attn_weights = self.attention(
            query=lstm_output, 
            key=lstm_output, 
            value=lstm_output,
            key_padding_mask=mask
        )
        
        # 4. Pooling global (seulement sur les steps valides)
        mask_expanded = mask.unsqueeze(-1)
        summed = torch.sum(attn_output * (~mask_expanded).float(), dim=1)
        context = summed / lengths.float().unsqueeze(-1)
        
        # 5. Transformation en vecteur latent
        z_mean = self.to_mean(context)
        z_logvar = self.to_logvar(context)
        
        return z_mean, z_logvar, attn_weights

class SafeAttentionDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, num_layers, num_heads=4):
        super().__init__()
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
        # Pas d'attention croisée avec l'encodeur pour éviter la fuite!
        
    def forward(self, z, lengths):
        batch_size = z.size(0)
        max_length = lengths.max()
        
        # Initialisation à partir du vecteur latent
        h0 = self.latent_to_hidden(z).unsqueeze(0).repeat(self.lstm.num_layers, 1, 1)
        c0 = torch.zeros_like(h0)
        
        # Séquence d'entrée nulle
        input_seq = torch.zeros(batch_size, max_length, self.lstm.input_size, device=z.device)
        
        # LSTM
        packed_input = pack_padded_sequence(input_seq, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input, (h0, c0))
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        return self.output_layer(output)

In [12]:
class LengthAwareLSTMEncoder(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, latent_dim: int, num_layers: int) -> None:
        super().__init__()
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=False, batch_first=True)
        self.encoder_linear_mean = nn.Linear(hidden_dim, latent_dim)
        self.encoder_linear_logvar = nn.Linear(hidden_dim, latent_dim)
        #self.dropout = nn.Dropout(p=0.5)

    def forward(self, x_padded: torch.Tensor, lengths: torch.Tensor, hidden: tuple = None) -> tuple[torch.Tensor]:
        if hidden is None:
            h0 = torch.zeros(self.encoder_lstm.num_layers, x_padded.size(0), self.encoder_lstm.hidden_size, device=x_padded.device)
            c0 = torch.zeros(self.encoder_lstm.num_layers, x_padded.size(0), self.encoder_lstm.hidden_size, device=x_padded.device)
            hidden = (h0, c0)

        packed_input = pack_padded_sequence(x_padded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, hidden = self.encoder_lstm(packed_input)
        #output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)

        last_hidden = hidden[0][-1]
        mean = self.encoder_linear_mean(last_hidden)
        logvar = self.encoder_linear_logvar(last_hidden)
        return mean, logvar, hidden

In [13]:
batch_size = 10
input_dim = 24
hidden_dim = 128
latent_dim = 24
num_layers = 2

n_time = 200
length_foo = torch.randint(low=50, high=n_time+1, size=(batch_size,))
seq_length = max(length_foo).item()
x_foo = torch.randn(batch_size, seq_length, input_dim)
encoder_foo = LengthAwareLSTMEncoder(input_dim, hidden_dim, latent_dim, num_layers)
mean_foo, logvar_foo, (h_final, c_final) = encoder_foo(x_foo, length_foo)


assert mean_foo.shape == (batch_size, latent_dim)
assert logvar_foo.shape == (batch_size, latent_dim)
assert h_final.shape == (num_layers, batch_size, hidden_dim)
assert c_final.shape == (num_layers, batch_size, hidden_dim)

In [14]:
class LengthAwareLSTMDecoder(nn.Module):
    def __init__(self, latent_dim: int, hidden_dim: int, output_dim: int, num_layers: int) -> None:
        super().__init__()
        self.decoder_linear_init = nn.Linear(latent_dim, hidden_dim * num_layers * 2)  # For hidden and cell states of each layer
        self.decoder_lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, bidirectional=False, batch_first=True)
        self.decoder_output_layer = nn.Linear(hidden_dim, output_dim)
        #self.dropout = nn.Dropout(p=0.5)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, z: torch.Tensor, lengths: torch.Tensor, hidden: tuple = None) -> torch.Tensor:
        batch_size = z.size(0)

        if hidden is None:
            init_states = self.decoder_linear_init(z)
            h0 = init_states[:, :self.hidden_dim * self.num_layers].reshape(self.num_layers, batch_size, self.hidden_dim)
            c0 = init_states[:, self.hidden_dim * self.num_layers:].reshape(self.num_layers, batch_size, self.hidden_dim)
            hidden = (h0, c0)

        max_length = torch.max(lengths)
        input_seq = torch.zeros(batch_size, max_length, self.hidden_dim, device=z.device)

        packed_input = pack_padded_sequence(input_seq, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, hidden_out = self.decoder_lstm(packed_input, hidden)

        transformed_data = self.decoder_output_layer(packed_output.data)
        output_packed = torch.nn.utils.rnn.PackedSequence(
            data=transformed_data, 
            batch_sizes=packed_output.batch_sizes,
            sorted_indices=packed_output.sorted_indices,
            unsorted_indices=packed_output.unsorted_indices
        )
        output, _ = pad_packed_sequence(output_packed, batch_first=True)

        return output, hidden_out

In [15]:
batch_size = 10
seq_length = 200
input_dim = 24
hidden_dim = 128
latent_dim = 24
num_layers = 2

z_foo = torch.randn(batch_size, latent_dim)
decoder_foo = LengthAwareLSTMDecoder(latent_dim, hidden_dim, input_dim, num_layers)
masked_output_foo, (h_out, c_out) = decoder_foo(z_foo, length_foo)

assert masked_output_foo.shape == (batch_size, torch.max(length_foo), input_dim)
assert h_out.shape == (num_layers, batch_size, hidden_dim)
assert c_out.shape == (num_layers, batch_size, hidden_dim)

In [16]:
class LSTMBetaVAE(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, latent_dim: int, lstm_num_layers: int, bptt_steps: int = 20) -> None:
        super().__init__()
        self.encoder = LengthAwareLSTMEncoder(input_dim, hidden_dim, latent_dim, lstm_num_layers)
        self.decoder = LengthAwareLSTMDecoder(latent_dim, hidden_dim, input_dim, lstm_num_layers)
        self.bptt_steps = bptt_steps
    
    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> tuple[torch.Tensor]:
        _, seq_len, _ = x.shape
        reconstructions = []
        all_z_means = []
        all_z_logvars = []
        
        # Initialization
        h_enc, c_enc = None, None
        h_dec, c_dec = None, None
        
        for start_idx in range(0, seq_len, self.bptt_steps):
            end_idx = min(start_idx + self.bptt_steps, seq_len)
            segment = x[:, start_idx:end_idx, :]
            
            if h_enc is not None:
                h_enc, c_enc = h_enc.detach(), c_enc.detach()
            if h_dec is not None:
                h_dec, c_dec = h_dec.detach(), c_dec.detach()
            
            z_mean_segment, z_logvar_segment, (h_enc, c_enc) = self.encoder(segment, lengths, (h_enc, c_enc))
            all_z_means.append(z_mean_segment)
            all_z_logvars.append(z_logvar_segment)
            z_segment = self.reparameterize(z_mean_segment, z_logvar_segment)
            
            x_reconstructed, (h_dec, c_dec) = self.decoder(z_segment, lengths, (h_dec, c_dec))
            reconstructions.append(x_reconstructed)

        z_means_all = torch.stack(all_z_means, dim=1)  # [batch, segments, latent_dim]
        z_logvars_all = torch.stack(all_z_logvars, dim=1)
        z_mean = torch.mean(z_means_all, dim=1)
        z_logvar = torch.mean(z_logvars_all, dim=1)
        reconstruction = torch.cat(reconstructions, dim=1)
        return reconstruction, z_mean, z_logvar

    def reparameterize(self, mean: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

In [17]:
batch_size = 10
input_dim = 24
hidden_dim = 128
latent_dim = 24
num_layers = 2

n_time = 200
length_foo = torch.randint(low=100, high=n_time+1, size=(batch_size,))
seq_length = max(length_foo).item()
x_foo = torch.randn(batch_size, seq_length, input_dim)
vae_foo = LSTMBetaVAE(input_dim, hidden_dim, latent_dim, num_layers)
output_foo, mean_foo, logvar_foo = vae_foo(x_foo, length_foo)


assert mean_foo.shape == (batch_size, latent_dim)
assert logvar_foo.shape == (batch_size, latent_dim)
assert output_foo.shape == (batch_size, torch.max(length_foo), input_dim)
assert output_foo.shape == x_foo.shape

RuntimeError: start (200) + length (10) exceeds dimension size (200).

In [18]:
def vae_loss_function(
    x_recon: torch.Tensor, 
    x: torch.Tensor, 
    z_mean: torch.Tensor, 
    z_logvar: torch.Tensor, 
    lengths: torch.Tensor, 
    beta: float = 1.0
    ) -> tuple[torch.Tensor]:

    _, seq_length, _ = x.shape

    mask = torch.arange(seq_length, device=x_recon.device)[None, :] < lengths[:, None]      # shape [batch_size, max_length]
    mask = mask.unsqueeze(-1).float()                                                       # shape [batch_size, max_length, 1]

    MSE = nn.functional.mse_loss(x_recon, x, reduction='none')
    MSE = (MSE * mask).sum(dim=(1,2))        # Mask application
    num_valid_steps = mask.sum(dim=(1,2))    # Normalizing factor
    MSE = torch.where(num_valid_steps > 0, MSE / num_valid_steps, torch.zeros_like(MSE))
    KLD = -0.5 * torch.sum(1 + z_logvar - z_mean.pow(2) - z_logvar.exp(), dim=1)

    MSE = torch.mean(MSE)
    KLD = torch.mean(KLD)
    TOTAL = MSE + beta * KLD

    return TOTAL, MSE, KLD

In [19]:
batch_size = 10
input_dim = 1
hidden_dim = 128
latent_dim = 24
num_layers = 1

n_time = 200
length_foo = torch.randint(low=100, high=n_time+1, size=(batch_size,))
seq_length = max(length_foo).item()
x_foo = torch.randn(batch_size, seq_length, input_dim)
x_recon_foo = torch.randn(batch_size, seq_length, input_dim)

z_mean_foo = torch.randn(batch_size, latent_dim)
z_logvar_foo = torch.randn(batch_size, latent_dim)

beta = 2.0

loss, loss_mse, loss_kld = vae_loss_function(
    x_foo, x_recon_foo, z_mean_foo, z_logvar_foo, length_foo, beta
)

print(loss)
print(loss_mse)
print(loss_kld)
assert loss.shape == torch.Size([])
assert loss_mse.shape == torch.Size([])
assert loss_kld.shape == torch.Size([])

tensor(42.6963)
tensor(2.0773)
tensor(20.3095)


In [20]:
def print_model_parameters(model, model_name="LSTMBetaVAE"):
    print(f"Model: {model_name}")
    print("=" * 60)
    
    total_params = 0
    for name, module in model.named_children():
        if hasattr(module, 'parameters'):
            module_params = sum(p.numel() for p in module.parameters() if p.requires_grad)
            print(f"{name}: {module_params:,} parameters")
            total_params += module_params
            
            print("-" * 40)
            for sub_name, sub_module in module.named_children():
                if hasattr(sub_module, 'parameters'):
                    sub_params = sum(p.numel() for p in sub_module.parameters() if p.requires_grad)
                    print(f"  ├─ {sub_name}: {sub_params:,} parameters")
                    
                    if hasattr(sub_module, 'named_parameters'):
                        print("  │   └─ Components:")
                        for param_name, param in sub_module.named_parameters():
                            if param.requires_grad:
                                print(f"  │      ├─ {param_name}: {param.numel():,} parameters "
                                      f"(shape: {tuple(param.shape)})")
            print("-" * 40)
            print()
    
    print("=" * 60)
    print(f"Total Trainable Parameters: {total_params:,}")
    
    print("\nArchitecture Details:")
    print("-" * 40)

    if hasattr(model, 'encoder'):
        encoder = model.encoder
        print("Encoder Structure:")
        for name, module in encoder.named_modules():
            if isinstance(module, torch.nn.LSTM):
                print(f"  LSTM: input_size={module.input_size}, "
                      f"hidden_size={module.hidden_size}, "
                      f"num_layers={module.num_layers}, "
                      f"bidirectional={module.bidirectional}")
            elif isinstance(module, torch.nn.Linear):
                print(f"  Linear: in_features={module.in_features}, "
                      f"out_features={module.out_features}")
    
    if hasattr(model, 'decoder'):
        decoder = model.decoder
        print("Decoder Structure:")
        for name, module in decoder.named_modules():
            if isinstance(module, torch.nn.LSTM):
                print(f"  LSTM: input_size={module.input_size}, "
                      f"hidden_size={module.hidden_size}, "
                      f"num_layers={module.num_layers}, "
                      f"bidirectional={module.bidirectional}")
            elif isinstance(module, torch.nn.Linear):
                print(f"  Linear: in_features={module.in_features}, "
                      f"out_features={module.out_features}")

In [21]:
vae_test = LSTMBetaVAE(
    input_dim=1,
    hidden_dim=32,
    latent_dim=16,
    lstm_num_layers=1
)
print_model_parameters(vae_test)

del vae_test
torch.cuda.empty_cache()

Model: LSTMBetaVAE
encoder: 5,536 parameters
----------------------------------------
  ├─ encoder_lstm: 4,480 parameters
  │   └─ Components:
  │      ├─ weight_ih_l0: 128 parameters (shape: (128, 1))
  │      ├─ weight_hh_l0: 4,096 parameters (shape: (128, 32))
  │      ├─ bias_ih_l0: 128 parameters (shape: (128,))
  │      ├─ bias_hh_l0: 128 parameters (shape: (128,))
  ├─ encoder_linear_mean: 528 parameters
  │   └─ Components:
  │      ├─ weight: 512 parameters (shape: (16, 32))
  │      ├─ bias: 16 parameters (shape: (16,))
  ├─ encoder_linear_logvar: 528 parameters
  │   └─ Components:
  │      ├─ weight: 512 parameters (shape: (16, 32))
  │      ├─ bias: 16 parameters (shape: (16,))
----------------------------------------

decoder: 9,569 parameters
----------------------------------------
  ├─ decoder_linear_init: 1,088 parameters
  │   └─ Components:
  │      ├─ weight: 1,024 parameters (shape: (64, 16))
  │      ├─ bias: 64 parameters (shape: (64,))
  ├─ decoder_lstm: 8,448 

## 3. Train loop

- On choisit d'éliminer les outliers par rapport à la densité de la reconstruction et non à la clusterisation de l'espace latent.

In [22]:
from torch.utils.data import Subset
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier

import gc
from torch.cuda.amp import autocast, GradScaler

In [23]:
from magnetics_diagnostic_analysis.ml_tools.train_callbacks import EarlyStopping, LRScheduling, GradientClipping, DropOutScheduling

In [24]:
def vae_reconstruction_error(
    x_recon: torch.Tensor, 
    x: torch.Tensor, 
    lengths: torch.Tensor, 
    ) -> tuple[torch.Tensor]:

    _, seq_length, _ = x.shape

    mask = torch.arange(seq_length, device=x_recon.device)[None, :] < lengths[:, None]      # shape [batch_size, max_length]
    mask = mask.unsqueeze(-1).float()                                                       # shape [batch_size, max_length, 1]

    mse = nn.functional.mse_loss(x_recon, x, reduction='none')
    mse = (mse * mask).sum(dim=(1,2))        # Mask application
    num_valid_steps = mask.sum(dim=(1,2))    # Normalizing factor
    mse = torch.where(num_valid_steps > 0, mse / num_valid_steps, torch.zeros_like(mse))

    return mse

In [25]:
def train_one_vae(model, optimizer, loader, full_loader, n_epochs_per_iter, device, verbose):
    # Training loop
    model.train()
    if verbose:
        print(f"{'Epoch':<40} {'Loss':<20} {'mse':<20} {'kld':<20}")
    for epoch in range(n_epochs_per_iter):
        total_loss = 0
        for batch_data, batch_lengths in tqdm(loader, desc=f"Training intermediate VAE", leave=False):
            batch_data = batch_data.to(device)
            batch_lengths = batch_lengths.to(device)

            optimizer.zero_grad()
            recon_batch, z_mean, z_logvar = model(batch_data, batch_lengths)
            loss, mse, kld = vae_loss_function(recon_batch, batch_data, z_mean, z_logvar, batch_lengths, beta=1.1)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            total_mse, total_kld = mse.item(), kld.item()

        epo, total_loss, total_mse, total_kld = f"Iteration {epoch + 1}/{n_epochs_per_iter}", total_loss/len(loader), total_mse/len(loader), total_kld/len(loader)
        if verbose:
            print(f"{epo:<40} {total_loss:<20} {total_mse:<20} {total_kld:<20}")
        gc.collect()
        torch.cuda.empty_cache()

    # VAE Evaluation
    model.eval()
    reconstruction_errors = []           # or len(current_subset)
    with torch.no_grad():
        for batch_data, batch_lengths in tqdm(full_loader, desc=f"Evaluating intermediate VAE", leave=False):
            batch_data = batch_data.to(device)
            batch_lengths = batch_lengths.to(device)
            
            recon_batch, _, _ = model(batch_data, batch_lengths)
            mse = vae_reconstruction_error(recon_batch, batch_data, batch_lengths)
            reconstruction_errors.append(mse.cpu())

    reconstruction_errors = torch.cat(reconstruction_errors).numpy()
    return reconstruction_errors

In [26]:
def find_threshold_kde(scores, alpha=0.05):
    kde = KernelDensity(kernel='gaussian', bandwidth='scott')
    kde.fit(scores.reshape(-1, 1))

    # Method 1: Based on gradient of density 
    # x = np.linspace(np.min(scores), np.max(scores), 1000)
    # log_dens = kde.score_samples(x.reshape(-1, 1))
    # density = np.exp(log_dens)
    # gradient = np.gradient(density)
    # inflection_point = x[np.argmin(gradient)]
    # Method 2: Percentile of density
    density_values = np.exp(kde.score_samples(scores.reshape(-1, 1)))
    threshold_density = np.percentile(density_values, alpha * 100)

    threshold = np.percentile(scores[density_values <= threshold_density], (1-alpha)*100)
    del kde
    return threshold

def detect_outliers_kde(scores, alpha=0.05):
    threshold = find_threshold_kde(scores, alpha)
    outlier_mask = scores > threshold
    outlier_indices = np.where(outlier_mask)[0]
    
    return outlier_indices, threshold

In [27]:
def train_final_vae(model, optimizer, loader, full_loader, n_epochs_per_iter, device, verbose):

    model.train()
    if verbose:
        print(f"{'Epoch':<40} {'Loss':<20} {'mse':<20} {'kld':<20}")
    for epoch in range(n_epochs_per_iter):
        total_loss = 0
        for batch_data, batch_lengths in tqdm(loader, desc="Training final VAE", leave=False):
            batch_data = batch_data.to(device)
            batch_lengths = batch_lengths.to(device)

            optimizer.zero_grad()
            recon_batch, z_mean, z_logvar = model(batch_data, batch_lengths)
            loss, mse, kld = vae_loss_function(recon_batch, batch_data, z_mean, z_logvar, batch_lengths, beta=1.1)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            total_mse, total_kld = mse.item(), kld.item()

        epo, total_loss, total_mse, total_kld = f"Iteration {epoch + 1}/{n_epochs_per_iter}", total_loss/len(loader), total_mse/len(loader), total_kld/len(loader)
        if verbose:
            print(f"{epo:<40} {total_loss:<20} {total_mse:<20} {total_kld:<20}")
        gc.collect()
        torch.cuda.empty_cache()

    # Latent features for all data
    model.eval()
    with torch.no_grad():
        z_mean_all = []
        for batch_data, batch_lengths in tqdm(full_loader, desc="Extracting latent features", leave=False):
            batch_data = batch_data.to(device)
            batch_lengths = batch_lengths.to(device)

            z_mean, _ = model.encoder(batch_data, batch_lengths)
            z_mean_all.append(z_mean.cpu().numpy())
        
        latent_features = np.concatenate(z_mean_all, axis=0)
    print("Latent features extracted for final VAE")
    return latent_features

In [28]:
def find_cluster_and_classify(latent_features, dbscan_eps, dbscan_min_samples, knn_n_neighbors):
    dbscan = DBSCAN(eps=dbscan_eps, min_samples=dbscan_min_samples)
    clusters = dbscan.fit_predict(latent_features)
    outlier_mask = clusters == -1
    binary_labels = (dbscan.labels_ != -1).astype(int)
    print("DBSCAN Clustering Completed")

    knn = KNeighborsClassifier(n_neighbors=knn_n_neighbors, weights='distance', metric='euclidean', algorithm='auto')
    knn.fit(latent_features, binary_labels)
    print("KNN Classifier Trained")

    del dbscan
    return knn, clusters, outlier_mask

In [29]:
def train_iterative_vae_pipeline(
    train_dataset: Dataset,
    n_iterations: int = 5,
    n_epochs_per_iter: int = 50,
    batch_size: int = 32,
    kde_percentile_rate: float = 0.05,
    dbscan_eps: float = 0.5,
    dbscan_min_samples: int = 5,
    knn_n_neighbors: int = 5,
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
) -> dict:
    
    # Model parameters
    sample_data, _ = train_dataset[0]
    print("Sample data shape:", sample_data.shape)
    input_dim = sample_data.shape[-1]
    print("Input dim:", input_dim)
    hidden_dim = 16
    latent_dim = 8
    lstm_layers = 1

    # Good and bad health indices initialization & model storage
    valid_indices = list(range(len(train_dataset)))
    all_anomaly_indices = np.array([], dtype=int)
    vae_models = []

    full_loader = DataLoader(dataset=train_dataset, 
                             batch_size=batch_size, 
                             shuffle=True, 
                             collate_fn=pad_sequences_smartly, 
                             drop_last=False,
                             pin_memory=False,
                             num_workers=0)

    for iteration in range(n_iterations):
        print(f"\nIteration {iteration + 1}/{n_iterations}")
        print(f"Training on {len(valid_indices)} samples...")

        # Data SubSet creation
        current_subset = Subset(train_dataset, valid_indices)
        train_loader = DataLoader(dataset=current_subset, 
                                  batch_size=batch_size, 
                                  shuffle=True, 
                                  collate_fn=pad_sequences_smartly, 
                                  drop_last=False, 
                                  pin_memory=False,
                                  num_workers=0)

        # VAE Training
        gc.collect()
        torch.cuda.empty_cache()
        vae = LSTMBetaVAE(input_dim, hidden_dim, latent_dim, lstm_layers).to(device)
        optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

        reconstruction_errors = train_one_vae(vae, optimizer, train_loader, full_loader, n_epochs_per_iter, device, verbose=True)

        # Outlier detection with KDE
        outlier_indices, threshold = detect_outliers_kde(reconstruction_errors, alpha=kde_percentile_rate)

        # Update anomaly indices
        all_anomaly_indices = np.unique(np.concatenate([all_anomaly_indices, outlier_indices]))
        valid_indices = list(np.setdiff1d(np.arange(len(train_dataset)), all_anomaly_indices))
        
        vae_models.append(vae.state_dict())
        print(f"New anomalies detected: {len(outlier_indices)}")
        print(f"Total anomalies: {len(all_anomaly_indices)}\n")

        # Delete cache and big variables
        del vae, optimizer, reconstruction_errors, current_subset, train_loader
        torch.cuda.empty_cache()
        gc.collect()



    # Last training phase
    print("Training final model...")
    final_subset = Subset(train_dataset, valid_indices)
    final_loader = DataLoader(final_subset, 
                              batch_size=batch_size, 
                              shuffle=True, 
                              collate_fn=pad_sequences_smartly, 
                              drop_last=False, 
                              pin_memory=False, 
                              num_workers=0)

    final_vae = LSTMBetaVAE(input_dim, hidden_dim, latent_dim, lstm_layers).to(device)
    optimizer = torch.optim.Adam(final_vae.parameters(), lr=1e-3)

    latent_features = train_final_vae(final_vae, optimizer, final_loader, full_loader, n_epochs_per_iter, device, verbose=True)

    # Final clustering on latent space on all train_dataset, with DBScan coupled with KNN
    knn, clusters, outlier_mask = find_cluster_and_classify(latent_features, dbscan_eps, dbscan_min_samples, knn_n_neighbors)

    del final_loader, final_subset, full_loader
    gc.collect()
    return {
        'final_vae': final_vae,
        'vae_models': vae_models,
        'knn': knn,
        'anomaly_indices': all_anomaly_indices,
        'latent_features': latent_features,
        'clusters': clusters,
        'outlier_mask': outlier_mask
    }

In [30]:
n_iterations = 1
n_epochs_per_iter = 2
batch_size = 1
kde_percentile_rate = 0.05
dbscan_eps = 0.5
dbscan_min_samples = 5
knn_n_neighbors = 10
device = config.DEVICE

results = train_iterative_vae_pipeline(
    train_dataset=train_set,
    n_iterations=n_iterations,
    n_epochs_per_iter=n_epochs_per_iter,
    batch_size=batch_size,
    kde_percentile_rate=kde_percentile_rate,
    dbscan_eps=dbscan_eps,
    dbscan_min_samples=dbscan_min_samples,
    knn_n_neighbors=knn_n_neighbors,
    device=device
)

Sample data shape: (174, 1)
Input dim: 1

Iteration 1/1
Training on 4588 samples...


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [31]:
def check_memory_usage():
    print(f"Mémoire GPU allouée: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Mémoire GPU réservée: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
    if torch.cuda.is_available():
        print(f"Mémoire GPU totale: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        print(f"Mémoire GPU disponible: {torch.cuda.get_device_properties(0).total_memory / 1024**3 - torch.cuda.memory_allocated() / 1024**3:.2f} GB")

check_memory_usage()

Mémoire GPU allouée: 0.00 MB
Mémoire GPU réservée: 0.00 MB
Mémoire GPU totale: 14.58 GB
Mémoire GPU disponible: 14.58 GB


In [32]:
final_vae = results['final_vae']
knn = results['knn']

NameError: name 'results' is not defined

In [ ]:
def evaluate_on_test(test_dataset, trained_vae, device):
    test_loader = DataLoader(
        dataset=test_dataset, 
        batch_size=32, 
        shuffle=False, 
        collate_fn=pad_sequences_smartly, 
        drop_last=False
    )
    trained_vae.eval()
    
    reconstruction_errors = []
    latent_features = []
    
    with torch.no_grad():
        for batch_data, batch_lengths in test_loader:
            batch_data = batch_data.to(device)
            batch_lengths = batch_lengths.to(device)
            
            recon_batch, z_mean, _ = trained_vae(batch_data, batch_lengths)
            # Store latent features
            latent_features.append(z_mean.cpu().numpy())

            mse = vae_reconstruction_error(recon_batch, batch_data, batch_lengths)
            # Store reconstruction errors
            reconstruction_errors.extend(mse.cpu().numpy())
    
    return np.array(reconstruction_errors), np.concatenate(latent_features, axis=0)

In [ ]:
def train_one_time(
    model: nn.Module,
    train_loader: DataLoader,
    n_epochs: int,
    optimizer: torch.optim.Optimizer,
    device: torch.device
):
    current_data = data.copy()
    anomaly_indices = np.array([], dtype=int)

    reconstruction_error_threshold_percentile = 95


    for epoch in range(n_epochs):
        model.train()
        for batch in train_loader:
            data, lengths = batch
            data = data.to(device)
            lengths = lengths.to(device)

            optimizer.zero_grad()
            output = model(data, lengths)
            loss = criterion(output, data)
            loss.backward()
            optimizer.step()




In [ ]:
def train(n_iterations: int):
    for i in range(n_iterations):
        model = VAE()
        train_one_time(model, train_loader, n_epochs, optimizer, device)

In [ ]:
from sklearn.ensemble import IsolationForest
